In [50]:
import pandas as pd
import re
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

## Getting Soup
def scraping(team):
    url='https://dondeverlo.com/futbol/equipo/{}'.format(team.lower())
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html,'html.parser')
    return soup

##next_match
def next_match(soup):
    matches = soup.findAll('h2',{"class":'Event__title'})
    next_match = matches[0].find("a").text
    next_match = next_match.lstrip().rstrip()
    return next_match

##team2
def team2(soup,team):
    matches = soup.findAll('h2',{"class":'Event__title'})
    next_match = matches[0].find("a").text
    next_match = next_match.lstrip().rstrip()
    nm2=re.sub("\s\s+" , " ",next_match)
    nm3=nm2.split(" - ",)
    nm3 = [w.replace('Alavés','Alaves') for w in nm3]
    nm3 = [w.replace('Celta de Vigo','Celta') for w in nm3]
    local=nm3[0].lower()
    visitor=nm3[1].lower()
    team=team.lower()
    
    print(nm3)
    if local==team:
        return visitor
    else:
        return local

##date_next_match
def date_next_match(soup):
    dates = soup.findAll('div',{"class":'Event__when'})
    date_next_match = dates[0].text
    date_next_match = date_next_match.lstrip().rstrip()
    return date_next_match

##competition_next_match
def competition_next_match(soup):
    competition = soup.findAll('span',{"class":'Event__championshipText'})
    competition_next_match = competition[0].text
    competition_next_match = competition_next_match.lstrip().rstrip()
    competition=competition_next_match.split("-")
    return competition[1].lstrip().rstrip()

##matchday_next_match
def matchday_next_match(soup):
    matchday = soup.findAll('div',{"class":'Event__description'})
    matchday_next_match = matchday[0].text
    matchday_next_match = matchday_next_match.lstrip().rstrip()
    return matchday_next_match

##summary scrapping next match
def printer(soup,team):
    return "El proximo partido de tu equipo es el {} contra el/la {} correspondiente a la {} de {}.".format(date_next_match(soup),team2(soup,team),matchday_next_match(soup),competition_next_match(soup))

## filtering dataframe
def dfFilter(team,team2):
    team2=team2.lower()
    print(team2)
    df=pd.read_csv("./input/FMEL_Dataset.csv")
    df['localTeam']=df['localTeam'].map(lambda x: x.lower() if isinstance(x,str) else x)
    df['visitorTeam']=df['visitorTeam'].map(lambda x: x.lower() if isinstance(x,str) else x)
    df_team1=df.replace("atletico de bilbao","athletic")
    df_team2=df_team1.replace("atletico de Madrid","atletico")
    df_team3=df_team2.replace("celta de vigo","celta")
    df_team4=df_team3.replace("alavés","alaves")
    df_team5=df_team4[(df['timestamp']>=1093644000) & ((df['localTeam']==(team)) | (df['localTeam']==(team2))) & ((df['visitorTeam']==(team2)) | (df['visitorTeam']==(team)))]
    
    print(df_team5.sort_values('timestamp', ascending=False).head(30))
    return df_team5.sort_values('timestamp', ascending=False)



##function to translate if your team wins, loses, or draws for each game
def result (row,team):
    if row['localGoals']==row['visitorGoals']:
        return 1
    if row['localTeam']==team and row['localGoals']>row['visitorGoals']:
        return 3
    if row['visitorTeam']==team and row['visitorGoals']>row['localGoals']:
        return 3
    else:
        return 0
    
##creating and appending new column
def summary(a,team,team2):
    a = a.value_counts()
    ##sign=a.value_counts()
    ##sign=df_team['result'].value_counts()
    return "{} y {} se han enfrentado {} veces en los últimos 20 años. {} ha ganado {} partidos, perdido {}, y empatado {}.".format(team,team2,a.sum(),team,a[3],a[0],a[1])


team="athletic"
team=team.lower()
soup=scraping(team)
next_match=next_match(soup)
team2=team2(soup,team)
team2=team2.lower()
print(team)
print(next_match)
print(team2)
print(date_next_match(soup))
print(competition_next_match(soup))
print(matchday_next_match(soup))

df_team=dfFilter(team,team2)
df_team["result"] = df_team.apply(lambda df_team: result(df_team,team), axis=1)

print(df_team)
print(summary(df_team['result'],team,team2))

['Osasuna', 'Athletic']
athletic
Osasuna - Athletic
osasuna
domingo 24 de nov. 14:00
LaLiga
Jornada 14
osasuna
Empty DataFrame
Columns: [id, season, division, round, localTeam, visitorTeam, localGoals, visitorGoals, date, timestamp]
Index: []


ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series